# Creating and Valuing a CDS Contract

Replicating Markit Pricing on 20 Aug 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.33 - This build:  11 Nov 2023 at 07:47 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Creating a CDS Contract

In [3]:
valuation_date = Date(20, 8, 2020)
effective_date = Date(21, 8, 2020)
maturity_date = Date(20, 6, 2025)
running_coupon = 0.05
notional = ONE_MILLION
long_protection = True

In [4]:
cds_contract = CDS(effective_date, maturity_date, running_coupon, notional, long_protection)

## Build Ibor Curve

In [5]:
settlement_date = Date(24, 8, 2020)

In [6]:
dcType = DayCountTypes.ACT_360
depo1 = IborDeposit(settlement_date, "1M", 0.001709, dcType)
depo2 = IborDeposit(settlement_date, "2M", 0.002123, dcType)
depo3 = IborDeposit(settlement_date, "3M", 0.002469, dcType)
depo4 = IborDeposit(settlement_date, "6M", 0.003045, dcType)
depo5 = IborDeposit(settlement_date, "12M", 0.004449, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

swapType = SwapTypes.PAY
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swap1 = IborSwap(settlement_date,"2Y", swapType, 0.002155,fixedFreq,dcType)
swap2 = IborSwap(settlement_date,"3Y", swapType, 0.002305,fixedFreq,dcType)
swap3 = IborSwap(settlement_date,"4Y", swapType, 0.002665,fixedFreq,dcType)
swap4 = IborSwap(settlement_date,"5Y", swapType, 0.003290,fixedFreq,dcType)
swap5 = IborSwap(settlement_date,"6Y", swapType, 0.004025,fixedFreq,dcType)
swap6 = IborSwap(settlement_date,"7Y", swapType, 0.004725,fixedFreq,dcType)
swap7 = IborSwap(settlement_date,"8Y", swapType, 0.005430,fixedFreq,dcType)
swap8 = IborSwap(settlement_date,"9Y", swapType, 0.006075,fixedFreq,dcType)
swap9 = IborSwap(settlement_date,"10Y", swapType, 0.006640,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9]

libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

# Build a CDS Curve

In [7]:
cdsSpread = 0.01

In [8]:
cds1 = CDS(settlement_date, "6M", cdsSpread)
cds2 = CDS(settlement_date, "1Y", cdsSpread)
cds3 = CDS(settlement_date, "2Y", cdsSpread)
cds4 = CDS(settlement_date, "3Y", cdsSpread)
cds5 = CDS(settlement_date, "4Y", cdsSpread)
cds6 = CDS(settlement_date, "5Y", cdsSpread)
cds7 = CDS(settlement_date, "7Y", cdsSpread)
cds8 = CDS(settlement_date, "10Y", cdsSpread)

In [9]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [10]:
recovery_rate = 0.40

In [11]:
issuer_curve = CDSCurve(valuation_date, cdss, libor_curve, recovery_rate)

In [12]:
print(issuer_curve)

OBJECT TYPE: CDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 0.5808219,  0.9901922
 1.0849315,  0.9817979
 2.0849315,  0.9986272
 3.0849315,  0.9818007
 4.0876712,  0.9651915
 5.0876712,  0.9490150
 7.0876712,  0.9174925
10.0904110,  0.8721194


In [13]:
cds_contract.print_payments(issuer_curve, 

TypeError: CDS.print_payments() missing 1 required positional argument: 'issuer_curve'

# Valuation

In [ ]:
spd = cds_contract.par_spread(settlement_date, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

NameError: name 'cds_contract' is not defined

In [ ]:
cds_contract.value_fast_approx(settlement_date, 0.004, 0.01, 0.40, 0.40)

(-195533.6454106767, -187200.31207734335, 533.6950180429558, 44.32197353540687)

In [ ]:
v = cds_contract.value(settlement_date, issuer_curve, recovery_rate)

In [ ]:
dirty_pv = v['dirty_pv'] 
clean_pv = v['clean_pv']

In [ ]:
print("DIRTY VALUE  %12.2f"% dirty_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

DIRTY VALUE    -199370.92
CLEAN VALUE   -191037.58


In [ ]:
cleanp = cds_contract.clean_price(settlement_date, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   119.039676


In [ ]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 60.0


In [ ]:
accrued_interest = cds_contract.accrued_interest()
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON -8333.333333333334


In [ ]:
prot_pv = cds_contract.protection_leg_pv(settlement_date, issuer_curve, recovery_rate)
print("PROTECTION_PV", prot_pv)

PROTECTION_PV 47813.122593011336


In [ ]:
premPV = cds_contract.premium_leg_pv(settlement_date, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 247184.03808119084


In [ ]:
cds_contract.risky_pv01(settlement_date, issuer_curve)

{'dirty_rpv01': 4.943680761623817, 'clean_rpv01': 4.77701409495715}

## Risk Measures

In [ ]:
cds_contract.credit_dv01(settlement_date, issuer_curve, recovery_rate)

514.3827831871749

In [ ]:
cds_contract.interest_dv01(settlement_date, issuer_curve, recovery_rate)

4.157797746738652

Copyright (c) 2020 Dominic O'Kane